## Código utilizado para análise de dados das reportagens: 
### - [Eleições 2024: Miguel Vaz é o prefeito mais rico eleito no 1º turno; conheça](https://g1.globo.com/politica/eleicoes/2024/eleicao-em-numeros/noticia/2024/10/06/eleicoes-2024-saiba-quem-e-o-prefeito-mais-rico-eleito-no-1o-turno.ghtml);
### - [Eleições 2024: conheça Sandro Mabel, o prefeito mais rico do Brasil](https://g1.globo.com/politica/eleicoes/2024/eleicao-em-numeros/noticia/2024/10/27/eleicoes-2024-conheca-sandro-mabel-o-prefeito-mais-rico-eleito-no-pais.ghtml) e 
### - [Eleições 2024: 17 dos 26 prefeitos eleitos em capitais são milionários, veja a lista dos mais ricos](https://g1.globo.com/politica/eleicoes/2024/eleicao-em-numeros/noticia/2024/10/27/eleicoes-2024-prefeitos-eleitos-em-capitais-sao-milionarios-veja-a-lista-dos-mais-ricos.ghtml). 

In [1]:
# Importa as bibliotecas necessárias para a análise
import pandas as pd
import numpy as np
import re
import numpy as np

In [2]:
# Me mostre tudo
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [3]:
# Lê os dados do arquivo CSV contendo informações eleitorais
df = pd.read_csv("data/bem_candidato_2024_BRASIL.csv", delimiter =';', encoding='latin-1')

In [4]:
df.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,SG_UF,SG_UE,NM_UE,SQ_CANDIDATO,NR_ORDEM_BEM_CANDIDATO,CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO,DS_BEM_CANDIDATO,VR_BEM_CANDIDATO,DT_ULT_ATUAL_BEM_CANDIDATO,HH_ULT_ATUAL_BEM_CANDIDATO
0,21/10/2024,12:31:55,2024,2,Eleição Ordinária,619,Eleições Municipais 2024,06/10/2024,SP,66478,LOUVEIRA,250002221033,6,21,"Veículo automotor terrestre: caminhão, automóv...",Fiat Uno ano 200/2007,"12000,00",06/10/2024,20:05:32
1,21/10/2024,12:31:55,2024,2,Eleição Ordinária,619,Eleições Municipais 2024,06/10/2024,SP,66478,LOUVEIRA,250002221034,1,12,Casa,"UMA CASA NA RUA JOAO VERARDO, 379, CASA 3, BAI...","120000,00",06/10/2024,20:05:32
2,21/10/2024,12:31:55,2024,2,Eleição Ordinária,619,Eleições Municipais 2024,06/10/2024,SP,66478,LOUVEIRA,250002221036,1,12,Casa,"50% de uma casa, situada na Alameda Ágata, 472...","600000,00",06/10/2024,20:05:32
3,21/10/2024,12:31:55,2024,2,Eleição Ordinária,619,Eleições Municipais 2024,06/10/2024,SP,66478,LOUVEIRA,250002221036,2,11,Apartamento,"50% de um apartamento, situado na Rua Ucilla L...","180000,00",06/10/2024,20:05:32
4,21/10/2024,12:31:55,2024,2,Eleição Ordinária,619,Eleições Municipais 2024,06/10/2024,SP,66478,LOUVEIRA,250002221036,3,11,Apartamento,"50% de um apartamento, situado na Rua Ucilla L...","169000,00",06/10/2024,20:05:32


In [5]:
# Corrigindo pontuação dos valores
df['VR_BEM_CANDIDATO'] = df['VR_BEM_CANDIDATO'].str.replace('.', '').str.replace(',', '.').str.strip()

# Em seguida, converta a coluna para o tipo float
df['VR_BEM_CANDIDATO'] = df['VR_BEM_CANDIDATO'].astype(float)

In [6]:
# União com o df de candidaturas do TSE
candidaturas = pd.read_csv("data/consulta_cand_2024_BRASIL.csv", delimiter =';', encoding='latin-1')

In [7]:
candidaturas.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,DS_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,NR_FEDERACAO,NM_FEDERACAO,SG_FEDERACAO,DS_COMPOSICAO_FEDERACAO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,SG_UF_NASCIMENTO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO
0,20/10/2024,19:30:33,2024,2,ELEIÇÃO ORDINÁRIA,1,619,Eleições Municipais 2024,06/10/2024,MUNICIPAL,AM,2534,MANACAPURU,13,VEREADOR,40002148408,10852,SANDRO FREIRE DO VALE,SANDRO HIROSHI,#NULO#,-4,NÃO DIVULGÁVEL,-3,#NE,PARTIDO ISOLADO,10,REPUBLICANOS,REPUBLICANOS,-1,#NULO#,#NULO#,#NULO#,40001730911,PARTIDO ISOLADO,REPUBLICANOS,AM,21/12/1982,21297562283,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,1,SOLTEIRO(A),3,PARDA,190,FOTÓGRAFO E ASSEMELHADOS,5,SUPLENTE
1,20/10/2024,19:30:33,2024,2,ELEIÇÃO ORDINÁRIA,1,619,Eleições Municipais 2024,06/10/2024,MUNICIPAL,MG,51896,SANTA RITA DO ITUETO,13,VEREADOR,130001957774,10212,ERICA KAROLINA CAETANO HENRIQUE,ERICA KAROLINA,#NULO#,-4,NÃO DIVULGÁVEL,-3,#NE,PARTIDO ISOLADO,10,REPUBLICANOS,REPUBLICANOS,-1,#NULO#,#NULO#,#NULO#,130001701435,PARTIDO ISOLADO,REPUBLICANOS,MG,08/02/2000,213072370281,4,FEMININO,5,ENSINO MÉDIO INCOMPLETO,1,SOLTEIRO(A),3,PARDA,194,FRENTISTA,5,SUPLENTE
2,20/10/2024,19:30:33,2024,2,ELEIÇÃO ORDINÁRIA,1,619,Eleições Municipais 2024,06/10/2024,MUNICIPAL,MG,51896,SANTA RITA DO ITUETO,13,VEREADOR,130001957769,10666,JOSÉ ALONSO BENINCA,ALONSO BENICÁ,#NULO#,-4,NÃO DIVULGÁVEL,-3,#NE,PARTIDO ISOLADO,10,REPUBLICANOS,REPUBLICANOS,-1,#NULO#,#NULO#,#NULO#,130001701435,PARTIDO ISOLADO,REPUBLICANOS,MG,04/11/1972,99720130221,2,MASCULINO,4,ENSINO FUNDAMENTAL COMPLETO,3,CASADO(A),1,BRANCA,278,VEREADOR,2,ELEITO POR QP
3,20/10/2024,19:30:33,2024,2,ELEIÇÃO ORDINÁRIA,1,619,Eleições Municipais 2024,06/10/2024,MUNICIPAL,RS,88412,SANTA MARIA,13,VEREADOR,210001882348,10888,EVERTON BITTENCOURTE MACHADO,EVERTON MACHADO,#NULO#,-4,NÃO DIVULGÁVEL,-3,#NE,PARTIDO ISOLADO,10,REPUBLICANOS,REPUBLICANOS,-1,#NULO#,#NULO#,#NULO#,210001687391,PARTIDO ISOLADO,REPUBLICANOS,RS,30/08/1981,79873570400,2,MASCULINO,4,ENSINO FUNDAMENTAL COMPLETO,3,CASADO(A),1,BRANCA,537,MOTORISTA PARTICULAR,5,SUPLENTE
4,20/10/2024,19:30:33,2024,2,ELEIÇÃO ORDINÁRIA,1,619,Eleições Municipais 2024,06/10/2024,MUNICIPAL,RS,88811,SÃO LUIZ GONZAGA,13,VEREADOR,210002070520,10777,MARIA SARITA GUEDES FERNANDES,SARITA GUEDES,#NULO#,-4,NÃO DIVULGÁVEL,-3,#NE,PARTIDO ISOLADO,10,REPUBLICANOS,REPUBLICANOS,-1,#NULO#,#NULO#,#NULO#,210001715189,PARTIDO ISOLADO,REPUBLICANOS,RS,29/09/1967,11835960469,4,FEMININO,8,SUPERIOR COMPLETO,3,CASADO(A),1,BRANCA,922,SERVIDOR PÚBLICO CIVIL APOSENTADO,4,NÃO ELEITO


In [8]:
df = pd.merge(candidaturas, df, on='SQ_CANDIDATO', how='inner')

In [9]:
df.head()

,DT_GERACAO_x,HH_GERACAO_x,ANO_ELEICAO_x,CD_TIPO_ELEICAO_x,NM_TIPO_ELEICAO_x,NR_TURNO,CD_ELEICAO_x,DS_ELEICAO_x,DT_ELEICAO_x,TP_ABRANGENCIA,SG_UF_x,SG_UE_x,NM_UE_x,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,DS_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,NR_FEDERACAO,NM_FEDERACAO,SG_FEDERACAO,DS_COMPOSICAO_FEDERACAO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,SG_UF_NASCIMENTO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,DT_GERACAO_y,HH_GERACAO_y,ANO_ELEICAO_y,CD_TIPO_ELEICAO_y,NM_TIPO_ELEICAO_y,CD_ELEICAO_y,DS_ELEICAO_y,DT_ELEICAO_y,SG_UF_y,SG_UE_y,NM_UE_y,NR_ORDEM_BEM_CANDIDATO,CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO,DS_BEM_CANDIDATO,VR_BEM_CANDIDATO,DT_ULT_ATUAL_BEM_CANDIDATO,HH_ULT_ATUAL_BEM_CANDIDATO
0,20/10/2024,19:30:33,2024,2,ELEIÇÃO ORDINÁRIA,1,619,Eleições Municipais 2024,06/10/2024,MUNICIPAL,MG,51896,SANTA RITA DO ITUETO,13,VEREADOR,130001957769,10666,JOSÉ ALONSO BENINCA,ALONSO BENICÁ,#NULO#,-4,NÃO DIVULGÁVEL,-3,#NE,PARTIDO ISOLADO,10,REPUBLICANOS,REPUBLICANOS,-1,#NULO#,#NULO#,#NULO#,130001701435,PARTIDO ISOLADO,REPUBLICANOS,MG,04/11/1972,99720130221,2,MASCULINO,4,ENSINO FUNDAMENTAL COMPLETO,3,CASADO(A),1,BRANCA,278,VEREADOR,2,ELEITO POR QP,21/10/2024,12:31:55,2024,2,Eleição Ordinária,619,Eleições Municipais 2024,06/10/2024,MG,51896,SANTA RITA DO ITUETO,1,21,"Veículo automotor terrestre: caminhão, automóv...",MOTO HONDA /CG 125 TITAN ANO 1999 COR VERMELHA,2000.0,06/10/2024,22:32:52
1,20/10/2024,19:30:33,2024,2,ELEIÇÃO ORDINÁRIA,1,619,Eleições Municipais 2024,06/10/2024,MUNICIPAL,MG,51896,SANTA RITA DO ITUETO,13,VEREADOR,130001957769,10666,JOSÉ ALONSO BENINCA,ALONSO BENICÁ,#NULO#,-4,NÃO DIVULGÁVEL,-3,#NE,PARTIDO ISOLADO,10,REPUBLICANOS,REPUBLICANOS,-1,#NULO#,#NULO#,#NULO#,130001701435,PARTIDO ISOLADO,REPUBLICANOS,MG,04/11/1972,99720130221,2,MASCULINO,4,ENSINO FUNDAMENTAL COMPLETO,3,CASADO(A),1,BRANCA,278,VEREADOR,2,ELEITO POR QP,21/10/2024,12:31:55,2024,2,Eleição Ordinária,619,Eleições Municipais 2024,06/10/2024,MG,51896,SANTA RITA DO ITUETO,2,21,"Veículo automotor terrestre: caminhão, automóv...",CAMINHAO M.BENZ/L 608D COR VERDE PLACA MQT1470,40000.0,06/10/2024,22:32:52
2,20/10/2024,19:30:33,2024,2,ELEIÇÃO ORDINÁRIA,1,619,Eleições Municipais 2024,06/10/2024,MUNICIPAL,MG,51896,SANTA RITA DO ITUETO,13,VEREADOR,130001957769,10666,JOSÉ ALONSO BENINCA,ALONSO BENICÁ,#NULO#,-4,NÃO DIVULGÁVEL,-3,#NE,PARTIDO ISOLADO,10,REPUBLICANOS,REPUBLICANOS,-1,#NULO#,#NULO#,#NULO#,130001701435,PARTIDO ISOLADO,REPUBLICANOS,MG,04/11/1972,99720130221,2,MASCULINO,4,ENSINO FUNDAMENTAL COMPLETO,3,CASADO(A),1,BRANCA,278,VEREADOR,2,ELEITO POR QP,21/10/2024,12:31:55,2024,2,Eleição Ordinária,619,Eleições Municipais 2024,06/10/2024,MG,51896,SANTA RITA DO ITUETO,3,21,"Veículo automotor terrestre: caminhão, automóv...",MOTO HONDA /CG FAN COR PRETA -ANO 2010 PLACA M...,5000.0,06/10/2024,22:32:52
3,20/10/2024,19:30:33,2024,2,ELEIÇÃO ORDINÁRIA,1,619,Eleições Municipais 2024,06/10/2024,MUNICIPAL,MG,51896,SANTA RITA DO ITUETO,13,VEREADOR,130001957769,10666,JOSÉ ALONSO BENINCA,ALONSO BENICÁ,#NULO#,-4,NÃO DIVULGÁVEL,-3,#NE,PARTIDO ISOLADO,10,REPUBLICANOS,REPUBLICANOS,-1,#NULO#,#NULO#,#NULO#,130001701435,PARTIDO ISOLADO,REPUBLICANOS,MG,04/11/1972,99720130221,2,MASCULINO,4,ENSINO FUNDAMENTAL COMPLETO,3,CASADO(A),1,BRANCA,278,VEREADOR,2,ELEITO POR QP,21/10/2024,12:31:55,2024,2,Eleição Ordinária,619,Eleições Municipais 2024,06/10/2024,MG,51896,SANTA RITA DO ITUETO,4,21,"Veículo automotor terrestre: caminhão, automóv...",CARRO VW/GOL -COR VERMELHA- PLACA /PUF1516,25000.0,06/10/2024,22:32:52
4,20/10/2024,19:30:33,2024,2,ELEIÇÃO ORDINÁRIA,1,619,Eleições Municipais 2024,06/10/2024,MUNICIPAL,RS,88412,SANTA MARIA,13,VEREADOR,210001882348,10888,EVERTON BITTENCOURTE MACHA

In [10]:
# Somando o total dos bens por candidatos 
df_grouped = df.groupby(
    ['NM_URNA_CANDIDATO', 'DS_CARGO', 'SG_PARTIDO', 'DS_GENERO', 'DS_COR_RACA', 'DS_OCUPACAO', 'SG_UF_x', 'NM_UE_x', 'DS_SIT_TOT_TURNO'], as_index=False
).agg({'VR_BEM_CANDIDATO': 'sum'})

In [11]:
df_grouped = df_grouped.rename(columns={'VR_BEM_CANDIDATO': 'TOTAL_BENS'})

In [12]:
df_grouped.head()

,NM_URNA_CANDIDATO,DS_CARGO,SG_PARTIDO,DS_GENERO,DS_COR_RACA,DS_OCUPACAO,SG_UF_x,NM_UE_x,DS_SIT_TOT_TURNO,TOTAL_BENS
0,EDILÂNIA DO MILHO VERDE,VEREADOR,PP,FEMININO,PARDA,"FEIRANTE, AMBULANTE E MASCATE",GO,NOVA GLÓRIA,SUPLENTE,100000.0
1,JC CESAR DE RITA DE BEG,VEREADOR,PSB,MASCULINO,PARDA,PESCADOR,MG,SÃO ROMÃO,NÃO ELEITO,10000.0
2,LEOCIR ZAMBAN,VEREADOR,MDB,MASCULINO,BRANCA,GERENTE,SC,CAXAMBU DO SUL,ELEITO POR QP,293649.7
3,MARCINHA FILHA DO JORDÃO,VEREADOR,AVANTE,FEMININO,BRANCA,OUTROS,SP,SERTÃOZINHO,SUPLENTE,50000.0
4,NEU ENFERMEIRO,VEREADOR,MDB,MASCULINO,PRETA,ENFERMEIRO,MG,IBIAÍ,SUPLENTE,136000.0


In [13]:
pd.options.display.float_format = '{:,.2f}'.format

### 💸 Bens dos candidatos a prefeito

In [14]:
prefeitos = df_grouped[df_grouped["DS_CARGO"] == "PREFEITO"]

In [15]:
prefeitos_ricos = prefeitos.nlargest(20, 'TOTAL_BENS')
prefeitos_ricos

,NM_URNA_CANDIDATO,DS_CARGO,SG_PARTIDO,DS_GENERO,DS_COR_RACA,DS_OCUPACAO,SG_UF_x,NM_UE_x,DS_SIT_TOT_TURNO,TOTAL_BENS
140112,JOÃO PINHEIRO,PREFEITO,PRTB,MASCULINO,BRANCA,EMPRESÁRIO,SP,MARÍLIA,NÃO ELEITO,"2,851,300,000.00"
49322,CLERI,PREFEITO,PDT,MASCULINO,BRANCA,EMPRESÁRIO,RS,DOIS IRMÃOS,NÃO ELEITO,"1,542,273,922.21"
227320,PROFESSORA DIANILEY,PREFEITO,AGIR,FEMININO,PRETA,PROFESSOR DE ENSINO FUNDAMENTAL,GO,COLINAS DO SUL,NÃO ELEITO,"1,401,171,000.00"
166847,MABEL,PREFEITO,UNIÃO,MASCULINO,BRANCA,EMPRESÁRIO,GO,GOIÂNIA,#NULO#,"313,405,916.29"
166848,MABEL,PREFEITO,UNIÃO,MASCULINO,BRANCA,EMPRESÁRIO,GO,GOIÂNIA,2º TURNO,"313,405,916.29"
99619,FRANCIS DA COMETA,PREFEITO,PL,MASCULINO,BRANCA,EMPRESÁRIO,MT,CÁCERES,NÃO ELEITO,"230,205,982.70"
188539,MIGUEL VAZ,PREFEITO,REPUBLICANOS,MASCULINO,BRANCA,EMPRESÁRIO,MT,LUCAS DO RIO VERDE,ELEITO,"219,715,755.20"
207040,PABLO MARÇAL,PREFEITO,PRTB,MASCULINO,BRANCA,EMPRESÁRIO,SP,SÃO PAULO,NÃO ELEITO,"169,503,058.17"
250055,RUY MUNIZ,PREFEITO,PSB,MASCULINO,BRANCA,MÉDICO,MG,MONTES CLAROS,NÃO ELEITO,"158,912,018.75"
195625,NEGO BEDRICHUK,PREFEITO,PT,MASCULINO,BRANCA,EMPRESÁRIO,SC,TRÊS BARRAS,NÃO ELEITO,"154,945,000.00"


In [16]:
vereadores = df_grouped[df_grouped["DS_CARGO"] == "VEREADOR"]

In [17]:
vereadores_ricos = vereadores.nlargest(20, 'TOTAL_BENS')
vereadores_ricos

,NM_URNA_CANDIDATO,DS_CARGO,SG_PARTIDO,DS_GENERO,DS_COR_RACA,DS_OCUPACAO,SG_UF_x,NM_UE_x,DS_SIT_TOT_TURNO,TOTAL_BENS
222916,"PROFESSOR CRÉSIO CAMPOS,",VEREADOR,PL,MASCULINO,BRANCA,PROFESSOR DE ENSINO SUPERIOR,MG,IPATINGA,SUPLENTE,"12,101,378,759.75"
24656,BAIÁ DO BELO MONTE,VEREADOR,PRD,MASCULINO,BRANCA,AGRICULTOR,PA,ANAPU,ELEITO POR QP,"2,415,265,000.00"
78339,EDIOMAR LINO,VEREADOR,REPUBLICANOS,MASCULINO,BRANCA,SERVIDOR PÚBLICO MUNICIPAL,TO,NOVO ACORDO,SUPLENTE,"1,500,180,000.00"
169368,MANOEL DO IZAIAS,VEREADOR,PT,MASCULINO,PRETA,AGRICULTOR,PA,ANAPU,SUPLENTE,"1,200,120,000.00"
138721,JOÃO DO BÁ,VEREADOR,PDT,MASCULINO,BRANCA,ODONTÓLOGO,GO,MOSSÂMEDES,ELEITO POR QP,"1,102,131,121.00"
103195,GEDEAN AMARO,VEREADOR,MDB,MASCULINO,PARDA,EMPRESÁRIO,AM,BERURI,ELEITO POR QP,"1,000,505,000.00"
72221,DR VINICIUS,VEREADOR,PSD,MASCULINO,PARDA,MÉDICO,PI,PARNAÍBA,SUPLENTE,"660,000,000.00"
133088,JONAS AFONSO,VEREADOR,PODE,MASCULINO,BRANCA,EMPRESÁRIO,SP,MOGI DAS CRUZES,SUPLENTE,"617,900,000.00"
94174,FATIMA CARMINO,VEREADOR,PT,FEMININO,BRANCA,PROFESSOR DE ENSINO FUNDAMENTAL,PI,PARNAÍBA,ELEITO POR QP,"415,293,804.00"
274838,URIL,VEREADOR,PDT,MASCULINO,PARDA,ENGENHEIRO,BA,TAPEROÁ,SUPLENTE,"300,039,000.00"
